In [ ]:
!uv pip install -U psycopg[binary,pool] sqlalchemy pandas matplotlib dotenv

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import dotenv_values

c_pg = dotenv_values("cred-pg.txt")

# None = unbegrenzt
pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", None)

In [ ]:
engine = create_engine(
    f'postgresql+psycopg://{c_pg["user"]}:{c_pg["password"]}@{c_pg["host"]}/{c_pg["dbname"]}', 
    connect_args = {
        'options': '-c search_path=${user},ugeobln,ugm,uinsta,umisc,umobility,usozmed,public', 
        'keepalives_idle': 120
    },
    pool_size=1, 
    max_overflow=0,
    execution_options={ 'isolation_level': 'AUTOCOMMIT' }
)

In [ ]:
with engine.connect() as con:
    sql = "select rolname  from pg_roles where rolname like 'u%'"
    df = pd.read_sql_query(text(sql), con)
df

In [ ]:
with engine.connect() as con:
    sql = """
    select schemaname, tablename 
    from pg_tables 
    where schemaname like 'u%'
    order by schemaname, tablename
    """
    df = pd.read_sql_query(text(sql), con)
df

In [ ]:
with engine.connect() as con:
    sql = """
    with
      spy as (
        select extract(year from sales_month) as jahr
        from retail_sales
      )
    select jahr, count(*) as anzahl
    from spy
    where jahr >= 2010
    group by jahr
    order by jahr
    """
    df = pd.read_sql_query(text(sql), con)
df.jahr = df.jahr.astype('int32')
df.set_index('jahr', inplace = True)
df

In [ ]:
df.plot.bar()